In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("test").getOrCreate()

# reading csv file

In [2]:
df=spark.read.csv(r"C:\Users\Shivaraj\OneDrive\Desktop\my work\assignment1\result.csv",header=True,inferSchema=True)
df.show(60,truncate=False)

+--------+-------+-----+
|NAME    |SUBJECT|MARKS|
+--------+-------+-----+
|Shivaraj|KAN    |56   |
|Shivaraj|ENG    |78   |
|Shivaraj|HINDI  |65   |
|Shivaraj|MATHS  |75   |
|Shivaraj|SCIENCE|66   |
|Shivaraj|SOCIAL |88   |
|Ajit    |KAN    |43   |
|Ajit    |ENG    |45   |
|Ajit    |HINDI  |88   |
|Ajit    |MATHS  |42   |
|Ajit    |SCIENCE|46   |
|Ajit    |SOCIAL |78   |
|Prahlad |KAN    |77   |
|Prahlad |ENG    |67   |
|Prahlad |HINDI  |62   |
|Prahlad |MATHS  |67   |
|Prahlad |SCIENCE|64   |
|Prahlad |SOCIAL |57   |
|Ratesh  |KAN    |25   |
|Ratesh  |ENG    |54   |
|Ratesh  |HINDI  |46   |
|Ratesh  |MATHS  |22   |
|Ratesh  |SCIENCE|22   |
|Ratesh  |SOCIAL |77   |
|Ansh    |KAN    |57   |
|Ansh    |ENG    |77   |
|Ansh    |HINDI  |43   |
|Ansh    |MATHS  |35   |
|Ansh    |SCIENCE|78   |
|Ansh    |SOCIAL |43   |
|Raju    |KAN    |22   |
|Raju    |ENG    |67   |
|Raju    |HINDI  |43   |
|Raju    |MATHS  |23   |
|Raju    |SCIENCE|43   |
|Raju    |SOCIAL |34   |
+--------+-------+-----+


# Pivoting the data

In [3]:
df1=df.groupBy("NAME").pivot("SUBJECT").max("MARKS")
df1.show()

+--------+---+-----+---+-----+-------+------+
|    NAME|ENG|HINDI|KAN|MATHS|SCIENCE|SOCIAL|
+--------+---+-----+---+-----+-------+------+
| Prahlad| 67|   62| 77|   67|     64|    57|
|  Ratesh| 54|   46| 25|   22|     22|    77|
|    Raju| 67|   43| 22|   23|     43|    34|
|Shivaraj| 78|   65| 56|   75|     66|    88|
|    Ajit| 45|   88| 43|   42|     46|    78|
|    Ansh| 77|   43| 57|   35|     78|    43|
+--------+---+-----+---+-----+-------+------+



# assign rank value using ntile(set vale 2 for pass 1 and fail 2)

In [4]:
from pyspark.sql.window import Window
windowSpec=Window.partitionBy("NAME").orderBy("ENG")
from pyspark.sql.functions import ntile

df2= df1.withColumn("pass_fail",ntile(2).over(windowSpec)). \
    select("NAME","ENG","HINDI","KAN","MATHS","SCIENCE","SOCIAL", \
    (df1.ENG>=35) & (df1.HINDI>=35) & (df1.KAN>=35) & (df1.MATHS>=35) & (df1.SCIENCE>=35) & (df1.SOCIAL>=35)). \
    withColumnRenamed("((((((ENG >= 35) AND (HINDI >= 35)) AND (KAN >= 35)) AND (MATHS >= 35)) AND (SCIENCE >= 35)) AND (SOCIAL >= 35))","Result")

df2.show(truncate=False)

+--------+---+-----+---+-----+-------+------+------+
|NAME    |ENG|HINDI|KAN|MATHS|SCIENCE|SOCIAL|Result|
+--------+---+-----+---+-----+-------+------+------+
|Prahlad |67 |62   |77 |67   |64     |57    |true  |
|Ratesh  |54 |46   |25 |22   |22     |77    |false |
|Raju    |67 |43   |22 |23   |43     |34    |false |
|Shivaraj|78 |65   |56 |75   |66     |88    |true  |
|Ajit    |45 |88   |43 |42   |46     |78    |true  |
|Ansh    |77 |43   |57 |35   |78     |43    |true  |
+--------+---+-----+---+-----+-------+------+------+



# counting total 

In [5]:
df3=df2.withColumn("Total",df2["ENG"]+df2["HINDI"]+df2["KAN"]+df2["MATHS"]+df2["SCIENCE"]+df2["SOCIAL"])
df3.show()

+--------+---+-----+---+-----+-------+------+------+-----+
|    NAME|ENG|HINDI|KAN|MATHS|SCIENCE|SOCIAL|Result|Total|
+--------+---+-----+---+-----+-------+------+------+-----+
| Prahlad| 67|   62| 77|   67|     64|    57|  true|  394|
|  Ratesh| 54|   46| 25|   22|     22|    77| false|  246|
|    Raju| 67|   43| 22|   23|     43|    34| false|  232|
|Shivaraj| 78|   65| 56|   75|     66|    88|  true|  428|
|    Ajit| 45|   88| 43|   42|     46|    78|  true|  342|
|    Ansh| 77|   43| 57|   35|     78|    43|  true|  333|
+--------+---+-----+---+-----+-------+------+------+-----+



In [6]:
from pyspark.sql.functions import *

df4=df3.withColumn("Final_Result",regexp_replace("Result","true","Pass")).na.replace("false",'Fail').drop("Result")
df4.show()

+--------+---+-----+---+-----+-------+------+-----+------------+
|    NAME|ENG|HINDI|KAN|MATHS|SCIENCE|SOCIAL|Total|Final_Result|
+--------+---+-----+---+-----+-------+------+-----+------------+
| Prahlad| 67|   62| 77|   67|     64|    57|  394|        Pass|
|  Ratesh| 54|   46| 25|   22|     22|    77|  246|        Fail|
|    Raju| 67|   43| 22|   23|     43|    34|  232|        Fail|
|Shivaraj| 78|   65| 56|   75|     66|    88|  428|        Pass|
|    Ajit| 45|   88| 43|   42|     46|    78|  342|        Pass|
|    Ansh| 77|   43| 57|   35|     78|    43|  333|        Pass|
+--------+---+-----+---+-----+-------+------+-----+------------+



# calculate percentage

In [7]:
df5=df4.withColumn('Percent',(col('Total')/600)*100) 


In [13]:
df5.show()

+--------+---+-----+---+-----+-------+------+-----+------------+------------------+
|    NAME|ENG|HINDI|KAN|MATHS|SCIENCE|SOCIAL|Total|Final_Result|           Percent|
+--------+---+-----+---+-----+-------+------+-----+------------+------------------+
| Prahlad| 67|   62| 77|   67|     64|    57|  394|        Pass| 65.66666666666666|
|  Ratesh| 54|   46| 25|   22|     22|    77|  246|        Fail|              41.0|
|    Raju| 67|   43| 22|   23|     43|    34|  232|        Fail|38.666666666666664|
|Shivaraj| 78|   65| 56|   75|     66|    88|  428|        Pass| 71.33333333333334|
|    Ajit| 45|   88| 43|   42|     46|    78|  342|        Pass| 56.99999999999999|
|    Ansh| 77|   43| 57|   35|     78|    43|  333|        Pass| 55.50000000000001|
+--------+---+-----+---+-----+-------+------+-----+------------+------------------+



# listing students result one below the other 

In [9]:
df5.select("*",round("Percent",2)).drop('Percent').withColumnRenamed("round(Percent, 2)","Percent").show(truncate=False,vertical=True )

-RECORD 0----------------
 NAME         | Prahlad  
 ENG          | 67       
 HINDI        | 62       
 KAN          | 77       
 MATHS        | 67       
 SCIENCE      | 64       
 SOCIAL       | 57       
 Total        | 394      
 Final_Result | Pass     
 Percent      | 65.67    
-RECORD 1----------------
 NAME         | Ratesh   
 ENG          | 54       
 HINDI        | 46       
 KAN          | 25       
 MATHS        | 22       
 SCIENCE      | 22       
 SOCIAL       | 77       
 Total        | 246      
 Final_Result | Fail     
 Percent      | 41.0     
-RECORD 2----------------
 NAME         | Raju     
 ENG          | 67       
 HINDI        | 43       
 KAN          | 22       
 MATHS        | 23       
 SCIENCE      | 43       
 SOCIAL       | 34       
 Total        | 232      
 Final_Result | Fail     
 Percent      | 38.67    
-RECORD 3----------------
 NAME         | Shivaraj 
 ENG          | 78       
 HINDI        | 65       
 KAN          | 56       
 MATHS      

In [10]:
df5.columns

['NAME',
 'ENG',
 'HINDI',
 'KAN',
 'MATHS',
 'SCIENCE',
 'SOCIAL',
 'Total',
 'Final_Result',
 'Percent']

# Class Toper Name

In [11]:
df5.summary("max").show()

+-------+--------+---+-----+---+-----+-------+------+-----+------------+-----------------+
|summary|    NAME|ENG|HINDI|KAN|MATHS|SCIENCE|SOCIAL|Total|Final_Result|          Percent|
+-------+--------+---+-----+---+-----+-------+------+-----+------------+-----------------+
|    max|Shivaraj| 78|   88| 77|   75|     78|    88|  428|        Pass|71.33333333333334|
+-------+--------+---+-----+---+-----+-------+------+-----+------------+-----------------+



# Total student names 

In [14]:
df5.createOrReplaceTempView('my_temp')
spark.sql("select count(*) from my_temp").show()

+--------+
|count(1)|
+--------+
|       6|
+--------+

